# Tipping Game Simulation

This simulation explores the interaction between customers and waiters, focusing on how service quality and tipping behavior affect satisfaction and earnings.

* Customer satisfaction and waiter earnings are determined by a predefined payoff matrix.
* Strategy waiters:
    * High previous earning: decrease service level (except basic)
    * Low previous earning: increase service level (except excellent)
* Strategy customers:
    * High previous satisfaction: decrease tip (except 5%)
    * Low previous satisfaction: increase tip (except 20%)

In [3]:
import mesa
import random

class Customer(mesa.Agent):
    def __init__(self, model, name):
        super().__init__(model)
        self.name = name
        self.satisfaction = 0
        self.tip = random.choice([5, 10, 20])

    def step(self, waiter, last_satisfaction):
        self.adjust_tip(last_satisfaction)
        self.evaluate_service(waiter)

    def adjust_tip(self, last_satisfaction):
        # Adjust the tip based on satisfaction
        if last_satisfaction < 3:
            if self.tip == 5:
                self.tip = 10
            elif self.tip == 10:
                self.tip = 20
            elif self.tip == 20:
                self.tip = 5
        elif last_satisfaction > 3:  # If satisfaction is high, reduce tip
            if self.tip == 20:
                self.tip = 10
            elif self.tip == 10:
                self.tip = 5
            elif self.tip == 5:
                self.tip = 10

    def evaluate_service(self, waiter):
        # Payoff matrix based on the service level and tip
        payoff_matrix = {
            "Basic": {5: (3, 3), 10: (1, 4), 20: (0, 5)},
            "Good": {5: (4, 1), 10: (3, 3), 20: (1, 4)},
            "Excellent": {5: (5, 0), 10: (4, 2), 20: (2, 3)}
        }
        # Update customer satisfaction and waiter's earnings
        self.satisfaction, waiter.earnings = payoff_matrix[waiter.service_level][self.tip]

class Waiter(mesa.Agent):
    def __init__(self, model, name):
        super().__init__(model)
        self.name = name
        self.earnings = 0  # Initialize earnings to 0
        self.service_level = random.choice(["Basic", "Good", "Excellent"])

    def step(self, last_earnings): 
        self.adjust_service_level(last_earnings)

    def adjust_service_level(self, last_earnings):
        # Service level adjustment based on earnings feedback from the customer
        if last_earnings < 3:  # If earnings were low, improve service
            if self.service_level == "Basic":
                self.service_level = "Good"
            elif self.service_level == "Good":
                self.service_level = "Excellent"
            elif self.service_level == "Excellent":
                self.service_level = "Basic"
        elif last_earnings > 3:  # If earnings were high, lower service
            if self.service_level == "Excellent":
                self.service_level = "Good"
            elif self.service_level == "Good":
                self.service_level = "Basic"
            elif self.service_level == "Basic":
                self.service_level = "Good"

class TippingGameModel(mesa.Model):
    def __init__(self, seed=None):
        super().__init__(seed=seed)
        self.customer = Customer(self, "Customer")
        self.waiter = Waiter(self, "Waiter")
        self.equilibrium_reached = False
        self.steps = 0

    def step(self):
        if not self.equilibrium_reached:
            self.steps +1

            # Here we don't reset the waiter's earnings
            self.waiter.step(self.waiter.earnings)  # Pass previous earnings to adjust service level
            self.customer.step(self.waiter, self.customer.satisfaction)  # Adjust tip and satisfaction

            # Update last satisfaction and earnings based on the current interaction
            last_satisfaction_customer = self.customer.satisfaction
            last_earnings_waiter = self.waiter.earnings

            self.print_step_results(last_satisfaction_customer, last_earnings_waiter)
            self.check_equilibrium(last_satisfaction_customer, last_earnings_waiter)

    def check_equilibrium(self, last_satisfaction_customer, last_earnings_waiter):
        # Define equilibrium condition (both customer and waiter have 3)
        if last_satisfaction_customer == 3 and last_earnings_waiter == 3:
            self.equilibrium_reached = True
            print("Nash equilibrium reached!")

    def print_step_results(self, last_satisfaction_customer, last_earnings_waiter):
        print(f"--- Step {self.steps} ---")
        print(f"{self.customer.name}: Satisfaction {last_satisfaction_customer}, Tip {self.customer.tip}%")
        print(f"{self.waiter.name}: Earnings {last_earnings_waiter}, Service Level {self.waiter.service_level}")
        print("-" * 20)

if __name__ == "__main__":
    model = TippingGameModel()
    while not model.equilibrium_reached and model.steps < 20:
        model.step()
    if model.steps >= 20:
        print("Maximum steps reached. Equilibrium not found.")

--- Step 1 ---
Customer: Satisfaction 0, Tip 20%
Waiter: Earnings 5, Service Level Basic
--------------------
--- Step 2 ---
Customer: Satisfaction 4, Tip 5%
Waiter: Earnings 1, Service Level Good
--------------------
--- Step 3 ---
Customer: Satisfaction 4, Tip 10%
Waiter: Earnings 2, Service Level Excellent
--------------------
--- Step 4 ---
Customer: Satisfaction 3, Tip 5%
Waiter: Earnings 3, Service Level Basic
--------------------
Nash equilibrium reached!
